In [1]:
from pathlib import Path
import numpy as np
import os
import sys
import pickle
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

import imblearn

from numpy import where
from matplotlib import pyplot
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.datasets import make_classification


## Load numpy arrays from the path where process icustays program saved those arrays

In [2]:
path = Path('/finalproject/npoutput').expanduser()

chart_data_np = np.load(path/'chart_all_data_04272109.npy')
demographic_data_np = np.load(path/'demographic_all_data_04272109.npy')
icd9_data_np = np.load(path/'icd9_all_data_04272109.npy')
y_np = np.load(path/'y_all_04272109.npy')



print(chart_data_np.shape)
print(demographic_data_np.shape)
print(icd9_data_np.shape)
print(y_np.shape)

(48854, 48, 59)
(48854, 14)
(48854, 17)
(48854,)


## Oversample positive icustays as there is data imbalance between positive and negative icustays in the original data

In [3]:
counter = Counter(y_np)
print(counter)

#chart_data_transposed = chart_data_np.transpose(2,0,1).reshape(3,-1)
chart_data_np_reshape = chart_data_np.reshape(len(chart_data_np),-1)
print(chart_data_np.shape)

print(y_np.shape)
oversample = SMOTE()
chart_data_np_reshape, y_np1 = oversample.fit_resample(chart_data_np_reshape, y_np)
# summarize the new class distribution
chart_data_np_1 = chart_data_np_reshape.reshape(len(chart_data_np_reshape),48,59)
counter = Counter(y_np1)
print(chart_data_np.shape)
print(y_np1.shape)
print(counter)


Counter({0.0: 39227, 1.0: 9627})
(48854, 48, 59)
(48854,)
(48854, 48, 59)
(78454,)
Counter({0.0: 39227, 1.0: 39227})


## Convert numpy arrays to pytorch arrays

In [4]:
chart_data = torch.from_numpy(chart_data_np_1).float()
demographic_data = torch.from_numpy(demographic_data_np).float()
icd9_data = torch.from_numpy(icd9_data_np).float()
y = torch.from_numpy(y_np1).float()


## Create CustomDataset class

In [5]:
#Prepare Dataset
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    
    def __init__(self, x, y):
        
        self.x = x
        self.y = y
    
    def __len__(self):
        
        return len(self.x)
    
    def __getitem__(self, index):
        
        return self.x[index], self.y[index]
        

dataset = CustomDataset(chart_data, y)



In [6]:
print(dataset)



## Collate function to collect batch data

In [7]:
def collate_fn(data):
    #print(len(data))
    x, y = zip(*data)
    return torch.stack(x), torch.stack(y).type(torch.LongTensor) 



## Split dataset into 80% training, 10% validation and 10% testing

In [8]:
from torch.utils.data.dataset import random_split

split = int(len(dataset)*0.8)

lengths = [split, len(dataset) - split]
train_dataset, rem_dataset = random_split(dataset, lengths)

val_test_split = int(len(rem_dataset)*0.5)

val_test_lengths = [val_test_split, len(rem_dataset) - val_test_split]
val_dataset, test_dataset = random_split(rem_dataset, val_test_lengths)


print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))
print("Length of test dataset:", len(test_dataset))

Length of train dataset: 62763
Length of val dataset: 7845
Length of test dataset: 7846


## Create dataloaders for the train, validation and test datasets

In [9]:
from torch.utils.data import DataLoader

def load_data(train_dataset, val_dataset, test_dataset, collate_fn):
    
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

    
    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset, test_dataset, collate_fn)

## Create Bi-directional LSTM model 

In [10]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        #self.lstm = nn.LSTM(input_size=58, hidden_size=58, bidirectional= True, batch_first=True, num_layers=3)
        self.lstm = nn.LSTM(input_size=59, hidden_size=59, bidirectional=True, batch_first=True, num_layers=1)
        self.tanh = nn.Tanh()
        self.lstmdropout = nn.Dropout()
        
        #self.fc1 = nn.Linear(8233, 256)
        #self.fc1 = nn.Linear(5473, 256)
        self.fc1 = nn.Linear(118, 50)
        self.fc2 = nn.Linear(50, 10)
        #self.fc2 = nn.Linear(116, 10)
        self.fc3 = nn.Linear(10, 2)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        batch_size = x.shape[0]
        #print('batch_size', batch_size)
        #print('x shape', x.shape)
        output,(h,c) = self.lstm(x)
        #print('output shape', output.shape)
        #print('h shape', h.shape)
        #print('h shape', h[0].shape,h[-1].shape)
        finaloutput = torch.concat([h[0],h[-1]],dim=1)
        #print('finaloutput shape',finaloutput.shape)
        
        #print(finaloutput.shape)
        finaloutput = self.lstmdropout(self.tanh(finaloutput))
        #print('after forward', finaloutput.shape)
        
            
        finaloutput = self.sigmoid(self.fc3(self.fc2(self.fc1(finaloutput))))
        #print('after sigmoid', finaloutput.shape)
        #for i in range(48):
        #    print(finaloutput[1,i,0],finaloutput[1,i,1])
        
        #print(finaloutput[0,1,0],finaloutput[0,1,1])
        #print(finaloutput[0,2,0],finaloutput[0,2,1])
        
        #print(finaloutput.shape)
        #finaloutput = torch.squeeze(finaloutput, 1)
        return finaloutput
    
model = LSTM() 

In [11]:
model

LSTM(
  (lstm): LSTM(59, 59, batch_first=True, bidirectional=True)
  (tanh): Tanh()
  (lstmdropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=118, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

## Print number of parameters used by the model

In [12]:
sum(p.numel() for p in model.parameters() if p.requires_grad) 

63122

## Create Adam optimizer and Cross Entropy Loss function

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

## Model evaluation function

In [14]:
from sklearn.metrics import *

def eval_model(model, val_loader):
    model.eval()
    y_pred = []
    y_true = torch.LongTensor()
    model.eval()
    for x, y in val_loader:
        y_hat = model(x)
        #y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
        y_hat = (y_hat > 0.5).int()
        #print(y_hat.shape)
        for i in range(len(y_hat)):
            if y_hat[i][0] == 1:
                y_pred.append(0)
            else:
                y_pred.append(1)
        #y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    
    y_pred_t = torch.Tensor(y_pred)
    #print(y_pred_t)
    #print(y_true)
    acc = accuracy_score(y_true, y_pred_t)
    precision = precision_score(y_true, y_pred_t)
    recall = recall_score(y_true, y_pred_t)
    roc_auc = roc_auc_score(y_true, y_pred_t)
    
    return acc, precision, recall, roc_auc

## Unit test the model

In [15]:
a, p, r, roc_auc = eval_model(model, val_loader)
print(a, p, r, roc_auc)

0.5523263224984066 0.5395909805977976 0.778562421185372 0.5498482209019644


## Training function 

In [16]:
#Training
def train(model, train_loader, val_loader, n_epochs):
    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, y in train_loader:
            #print(len(x))
            optimizer.zero_grad()
            y_hat = model(x)
            #print(y_hat)
            #print(y)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        a, p, r, roc_auc = eval_model(model, val_loader)
        print('Epoch: {} \t Validation a: {:.3f}, p:{:.3f}, r: {:.3f}, roc_auc: {:.3f}'
              .format(epoch+1, a, p, r, roc_auc))
        

## Call the training function for 10 epochs

In [17]:
import time

start_time_nanosec = time.time_ns()

n_epochs = 10
train(model, train_loader, val_loader, n_epochs)

end_time_nanosec = time.time_ns()
print('end_time_nanosec', end_time_nanosec)


print('total time in seconds', (end_time_nanosec - start_time_nanosec)/1000000000)

Epoch: 1 	 Training Loss: 0.593112
Epoch: 1 	 Validation a: 0.771, p:0.826, r: 0.693, roc_auc: 0.772
Epoch: 2 	 Training Loss: 0.537764
Epoch: 2 	 Validation a: 0.783, p:0.841, r: 0.703, roc_auc: 0.784
Epoch: 3 	 Training Loss: 0.543931
Epoch: 3 	 Validation a: 0.759, p:0.766, r: 0.754, roc_auc: 0.759
Epoch: 4 	 Training Loss: 0.522419
Epoch: 4 	 Validation a: 0.796, p:0.876, r: 0.694, roc_auc: 0.797
Epoch: 5 	 Training Loss: 0.513530
Epoch: 5 	 Validation a: 0.801, p:0.912, r: 0.672, roc_auc: 0.803
Epoch: 6 	 Training Loss: 0.510784
Epoch: 6 	 Validation a: 0.802, p:0.924, r: 0.663, roc_auc: 0.804
Epoch: 7 	 Training Loss: 0.502865
Epoch: 7 	 Validation a: 0.817, p:0.936, r: 0.685, roc_auc: 0.818
Epoch: 8 	 Training Loss: 0.513423
Epoch: 8 	 Validation a: 0.808, p:0.915, r: 0.683, roc_auc: 0.809
Epoch: 9 	 Training Loss: 0.499708
Epoch: 9 	 Validation a: 0.815, p:0.974, r: 0.651, roc_auc: 0.817
Epoch: 10 	 Training Loss: 0.491761
Epoch: 10 	 Validation a: 0.815, p:0.984, r: 0.644, roc

## Evaluate the model on the testing data

In [18]:
a, p, r, roc_auc = eval_model(model, test_loader)
print('Epoch: {} \t Testing a: {:.3f}, p:{:.3f}, r: {:.3f}, roc_auc: {:.3f}'
              .format(1, a, p, r, roc_auc))

Epoch: 1 	 Testing a: 0.820, p:0.980, r: 0.649, roc_auc: 0.818
